In [1]:
!pip install -U transformers bitsandbytes

In [2]:
# !unzip GSM_Symbolic_Robustness.zip

In [3]:
%cd /content/GSM_Symbolic_Robustness/dataset

/content/GSM_Symbolic_Robustness/dataset


In [4]:
config = {
    'batch_size': 4,
}

In [5]:
# import importlib
# importlib.reload(datasets)

In [6]:
import datasets
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
val_dataset = datasets.GSM8K_Val_Dataset('gsm8k/test.jsonl', tokenizer, tiny=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import transformers
from transformers import BitsAndBytesConfig
# load in quantization config
q_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "microsoft/Phi-3.5-mini-instruct"

token = "INSERT HF TOKEN HERE"

model = transformers.AutoModelForCausalLM.from_pretrained(model_name, token=token, quantization_config=q_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [9]:
model.to(device)
model.train()

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3SdpaAttention(
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear4bit(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3LongRoPEScaledRotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear4bit(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi

In [ ]:
import re
import numpy as np
def extract_a(text):
  regex = re.compile(r"#### (\-?[0-9\.\,]+)")
  model_answer = regex.search(text)
  if model_answer:
    answer = model_answer.group(1)
    answer = answer.replace(",", "")
    answer = answer.replace(" ", "")
    return answer
  else:
    return "No Valid Answer Found"

def get_accuracy(outputs, a):
  pred_a = [extract_a(out) for out in outputs]
  a = [extract_a(ans) for ans in a]
  error_rate = np.mean(np.array(pred_a) == "No Valid Answer Found")
  return np.mean(np.array(pred_a) == np.array(a)), error_rate

In [ ]:
from tqdm import tqdm
acc = 0
error_rate = 0
pbar = tqdm(val_dataloader)
num_batches_done = 0
for q, q_mask, a in pbar:
    # move to device
    q = torch.stack(q, dim=1).to(device)
    q_mask = torch.stack(q_mask, dim=1).to(device)
    # do inference
    pred_a = model.generate(
      input_ids=q,
      attention_mask=q_mask,
      max_new_tokens=256
    )
    outputs = tokenizer.batch_decode(pred_a[:, q.shape[-1]:], skip_special_tokens=True)
    new_acc, new_error_rate = get_accuracy(outputs, a)
    acc += new_acc
    error_rate += new_error_rate
    num_batches_done += 1
    # do display
    pbar.set_postfix(accuracy=f"{(acc/num_batches_done):.2f}%", loss=f"{(error_rate/num_batches_done):.2f}")
    pbar.update(1)
acc /= len(val_dataloader)
error_rate /= len(val_dataloader)
print(f"\nAccuracy: {acc}")
print(f"Error Rate: {error_rate}")

  0%|          | 0/25 [00:00<?, ?it/s]